# Electrify_Clusters

### All the necessary Python imports

In [1]:
%matplotlib inline
from pathlib import Path
from openelec import electrify

%load_ext autoreload
%autoreload 2

### Enter all input data here

In [2]:
folder_input = Path('/home/chris/Documents/Code/openelec_app/openelec_app/static/lesotho')

clusters_file = folder_input / 'clusters.geojson' # must be polygons with attributes pop_sum, area_m2, grid_dist
#clusters_out = folder_input / 'clusters_out1.gpkg'
#network_out = folder_input / 'network_out1.gpkg'

grid_dist_connected = 1000  # clusters within this distance of grid are considered connected

minimum_pop = 100 # exclude any population below this

# off-grid costs
demand_per_person_kwh_month = 6 # 6kWh/month = MTF Tier 2
demand_per_person_kw_peak = demand_per_person_kwh_month / (4*30)  # 130 4hours/day*30days/month based on MTF numbers, should use a real demand curve
mg_gen_cost_per_kw = 4000
mg_cost_per_m2 = 2

# grid costs
cost_wire_per_m = 50
grid_cost_per_m2 = 2

### Read in the clusters file, convert to desired CRS (ostensibly better for distances) and convert to points, filter on population along the way

In [3]:
clusters = electrify.load_clusters(clusters_file, grid_dist_connected=grid_dist_connected,
                                   minimum_pop=minimum_pop)

In [6]:
clusters.gdp.max()

1694.0

### We then take all the clusters and calculate the optimum network that connects them all together. The ML model returns T_x and T_y containing the start and end points of each new arc created

In [4]:
network, nodes = electrify.create_network(clusters)

### Then we're ready to calculate the optimum grid extension.
This is done by expanding out from each already connected node, finding the optimum connection of nearby nodes. This is then compared to the off-grid cost and if better, these nodes are marked as connected. Then the loop continues until no new connections are found.

In [5]:
network, nodes = electrify.run_model(network,
                           nodes,
                           demand_per_person_kw_peak=demand_per_person_kw_peak,
                           mg_gen_cost_per_kw=mg_gen_cost_per_kw,
                           mg_cost_per_m2=mg_cost_per_m2,
                           cost_wire_per_m=cost_wire_per_m,
                           grid_cost_per_m2=grid_cost_per_m2)

### And then do a join to get the results back into a polygon shapefile

In [6]:
network_gdf, clusters_joined = electrify.spatialise(network, nodes, clusters)

In [18]:
network_gdf

,enabled,existing,i,len,ne,ns,xe,xs,ye,ys,geometry
0,1,0,0,2073,2,1321,281583,279939,-3342937,-3341673,LINESTRING (27.73809009251372 -29.160592657992...
9,1,0,9,3209,879,11,308943,305801,-3438943,-3439597,LINESTRING (27.99226641778219 -30.092733116806...
10,1,0,10,2322,13,1352,383345,385574,-3400249,-3399596,LINESTRING (28.77221658638564 -29.709769295614...
11,1,0,11,4722,189,14,326126,329860,-3412250,-3415142,LINESTRING (28.22735505548867 -29.858806882650...
12,1,0,12,1528,855,15,402155,401573,-3359044,-3357631,LINESTRING (28.9280542278586 -29.3103609536450...
13,1,0,13,1917,16,622,304920,303103,-3335165,-3335779,LINESTRING (27.96458476583038 -29.104457901746...
16,1,0,16,2048,1029,18,255363,253366,-3445246,-3445701,LINESTRING (27.47925276197781 -30.151710321804...
17,1,0,17,2074,19,338,281818,283697,-3360842,-3359963,LINESTRING (27.77505843007755 -29.334100437975...
19,1,0,19,2189,23,1351,370071,368309,-3377828,-3379128,LINESTRING (28.6029997585836 -29.5151571375686...
20,1,0,20,2546,1185,24,261468,259487,-3466456,-3468056,LINESTRING (27.53938471872659 -30.365782214005...


In [17]:
clusters_joined

,fid,area,pop,ntl,travel,gdp,grid_dist,geometry,conn_start,conn_end,og_cost,type
0,27,1.806602e+08,327483.875000,255.0,24.702128,80.0,0.000000,"POLYGON ((27.47560113505318 -29.4292412062753,...",1,1,0.0,orig
1,1673,1.676341e+07,39995.414062,255.0,45.250000,500.0,0.000000,"POLYGON ((27.9227161787829 -28.92463321907169,...",1,1,0.0,orig
2,18,3.165540e+07,36087.843750,255.0,38.666667,94.0,2692.582404,POLYGON ((27.74661093564346 -29.21621181827337...,0,1,70528200.0,new
3,1020,1.828877e+07,29947.791016,255.0,70.434783,253.0,0.000000,POLYGON ((27.27837140500067 -29.79178050530675...,1,1,0.0,orig
4,1509,1.420920e+07,26214.005859,255.0,1313.900000,13.5,0.000000,POLYGON ((28.87278596195058 -29.01398911424996...,1,1,0.0,orig
5,13,1.022603e+07,22324.906250,141.0,520.500000,34.5,0.000000,"POLYGON ((29.0661930914926 -29.29714315701265,...",1,1,0.0,orig
6,12,2.269124e+07,22291.720703,251.0,53.967742,60.0,250.000000,POLYGON ((27.75167289337718 -29.48678808705363...,1,1,0.0,orig
7,26,1.751480e+07,21688.113281,255.0,51.107143,174.0,0.000000,POLYGON ((28.04631458222579 -28.88634676534589...,1,1,0.0,orig
8,1421,4.188313e+06,20240.160156,223.0,1733.666667,17.0,15305.227865,POLYGON ((28.63610445549123 -29.02462866817705...,0,1,12424626.0,new
9,1654,1.849610e+07,19605.476562,255.0,75.680000,169.0,1030.776406,POLYGON ((28.22482308974107 -28.78852786102268...,0,1,40913206.0,new


In [8]:
#clusters_joined.to_file(str(clusters_out), driver='GPKG')
#network_gdf.to_file(str(network_out), driver='GPKG')

### And display some summary results

In [22]:
new_conns = clusters_joined.loc[clusters_joined['type'] == 'new']
og = clusters_joined.loc[clusters_joined['type'] == 'og']
orig = clusters_joined.loc[clusters_joined['type'] == 'orig']
cost = og['og_cost'].sum() + cost_wire_per_m * network_gdf['len'].sum() + grid_cost_per_m2 * new_conns['area'].sum()

total_modelled_pop = clusters['pop'].sum()
urban_elec_rate = 0.6
currently_electrified = orig['pop'].sum() * urban_elec_rate
new_conn_pop = new_conns['pop'].sum()
off_grid_pop = og['pop'].sum()

print(f'{len(new_conns)} connected')
print(f'{len(og)} off-grid')
print()
print(f'Cost ${cost:,.0f}')
print()
print(f'Modelled pop: {total_modelled_pop:,.0f}')
print(f'Currently electrified: {currently_electrified:,.0f}')
print(f'New connections: {new_conn_pop:,.0f}')
print(f'Off-grid connections {off_grid_pop:,.0f}')

767 connected
736 off-grid

Cost $3,426,290,177

Modelled pop: 2,041,503
Currently electrified: 471,170
New connections: 1,038,907
Off-grid connections 217,314
